In [1]:
#   %pip install opencv-python numpy pillow torch torchvision

In [2]:
#  !git clone https://github.com/xuebinqin/U-2-Net.git

In [3]:
%cd U-2-Net

c:\Users\rajki\OneDrive\Desktop\Background removal\U-2-Net


In [4]:
import os
import torch
import numpy as np

In [5]:
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable
from model.u2net import U2NETP 

# Load Model

In [6]:
def load_model(model_path='u2netp.pth'):
    net=U2NETP(3,1)
    net.load_state_dict(torch.load(model_path,map_location='cpu'))
    net.eval()
    return net

# Preprocessing

In [7]:
def preprocess(image_path):
    image=Image.open(image_path).convert("RGB").resize((480,480))
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    image_tensor=transform(image).unsqueeze(0)
    return image,image_tensor


# Postprocessing

In [8]:
def apply_mask(org_image,pred_mask,save_path='output.png'):
    pred_mask=pred_mask.squeeze().cpu().data.numpy()
    pred_mask=(pred_mask-pred_mask.min()) / (pred_mask.max()-pred_mask.min())
    pred_mask=np.array(pred_mask*255).astype(np.uint8)
    pred_mask=Image.fromarray(pred_mask).resize(org_image.size)
    # Mask apply
    org_np=np.array(org_image)
    mask_np=np.array(pred_mask) / 255.0
    mask_np=np.expand_dims(mask_np,axis=2)

    result=org_np*mask_np+255*(1-mask_np)
    result=result.astype(np.uint8)
    Image.fromarray(result).save(save_path)
    print(f"✅ Background removed image saved to: {save_path}")
  

# Main Function


In [9]:
def remove_background(image_path,output_path='cleaned_img.png'):
    model=load_model()
    original_image,tensor_image=preprocess(image_path)
    with torch.no_grad():
        d1, *_ = model(tensor_image)
    apply_mask(original_image,d1,output_path)

In [11]:
if __name__ == "__main__":
    image_path = r"C:\Users\rajki\OneDrive\Desktop\Background removal\shivansh-singh-1FbvP_0IeGA-unsplash.jpg"
    remove_background(image_path)

C:\Users\rajki\AppData\Local\Temp\ipykernel_18908\1126421711.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(model_path,map_location='cpu'

✅ Background removed image saved to: cleaned_img.png
